# データ分析のためのデータ形式
### データ分析に向いているデータの形式は以下のようなもの
   - 表形式になっている
   - １つの行が１つのデータ > 行方向にデータが並んでいる
   - １つの列がデータの1項目 > 列方向にデータの項目（属性）が並んでいる
   - データの追加、削除が自由にできる
   - データ項目の追加、削除が自由にできる
   - 分析に応じてデータを部分的に使うことができる
   
### 例：RESASから大量にデータを取り出して表形式にしてみる
- 以下のプログラムを実行して都道府県名を入力すると、2012,2013,2014,2015,2016,2017,2018,2019　の国籍別外国人訪問者数が四半期別に取得できる
- 取得したデータを表形式にしている。
- 列には見出しがついていて、データの意味が分かりやすくなっている

In [3]:
import json
import urllib.parse
import urllib.request
import pprint
import pandas as pd
import codecs

api_key={"X-API-KEY":"Bi0mpfnfqrVq6XttdXknIb8tTZTbxrLlebbRpBYg"}
url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'

req = urllib.request.Request(url, headers=api_key)

with urllib.request.urlopen(req) as response:
    pref_data = response.read()

pref_d = json.loads(pref_data.decode())

pref_code ={}
for p_data in pref_d["result"]:
    pref_code[p_data["prefName"]]= p_data["prefCode"]
print(pref_code)

p_name = input('都道府県名:')
url_base = 'https://opendata.resas-portal.go.jp/api/v1/tourism/foreigners/forFrom'
foreign_visitors=[]
years = [2012,2013,2014,2015,2016,2017,2018,2019]

p_code = pref_code[p_name]

for year in years:
    p = {'year':year,'prefCode':p_code, 'purpose': 1}
    url = url_base + '?' + urllib.parse.urlencode(p)
   
    req = urllib.request.Request(url, headers=api_key)

    with urllib.request.urlopen(req) as response:
        data = response.read()
    d = json.loads(data.decode())

    #pprint.pprint(d)

    #for foreigners in d['result']['changes']:
    #    print(foreigners)



    for foreigners in d['result']['changes']:
        #print(foreigners['countryName'], foreigners['data'])
        annual_value=0
        quarter_val = [0]*4
        for i,f in enumerate(foreigners['data']):
           annual_value += f['value']
           quarter_val[i]=f['value']
        #print(foreigners['countryName'], annual_value)
        foreign_visitors.append([p_name, year, foreigners['countryName'], annual_value]+quarter_val)


f_visitors_df = pd.DataFrame(foreign_visitors, columns=['都道府県名','年','国籍','年間訪問者数','第1四半期','第2四半期','第3四半期','第4四半期'])
f_visitors_df


{'北海道': 1, '青森県': 2, '岩手県': 3, '宮城県': 4, '秋田県': 5, '山形県': 6, '福島県': 7, '茨城県': 8, '栃木県': 9, '群馬県': 10, '埼玉県': 11, '千葉県': 12, '東京都': 13, '神奈川県': 14, '新潟県': 15, '富山県': 16, '石川県': 17, '福井県': 18, '山梨県': 19, '長野県': 20, '岐阜県': 21, '静岡県': 22, '愛知県': 23, '三重県': 24, '滋賀県': 25, '京都府': 26, '大阪府': 27, '兵庫県': 28, '奈良県': 29, '和歌山県': 30, '鳥取県': 31, '島根県': 32, '岡山県': 33, '広島県': 34, '山口県': 35, '徳島県': 36, '香川県': 37, '愛媛県': 38, '高知県': 39, '福岡県': 40, '佐賀県': 41, '長崎県': 42, '熊本県': 43, '大分県': 44, '宮崎県': 45, '鹿児島県': 46, '沖縄県': 47}


都道府県名: 東京都


,都道府県名,年,国籍,年間訪問者数,第1四半期,第2四半期,第3四半期,第4四半期
0,東京都,2012,韓国,730331,195991,157109,194386,182845
1,東京都,2012,中国,845709,202204,240194,326725,76586
2,東京都,2012,台湾,615463,123455,169409,180846,141753
3,東京都,2012,香港,251594,56710,65129,66242,63513
4,東京都,2012,タイ,144820,30663,43853,22718,47586
...,...,...,...,...,...,...,...,...
143,東京都,2019,イタリア,140441,20329,43093,44479,32540
144,東京都,2019,ロシア,90033,16033,26013,22410,25577
145,東京都,2019,カナダ,314216,64276,85231,74891,89818
146,東京都,2019,米国,1313048,262089,404908,315981,330070


- 上記のような表形式のデータをDataFrameと呼ぶ
- DataFrameはデータの分析が容易になるように柔軟な構造となっている。以下にその例を示す

例1. 米国からの本門者データだけを取り出す

In [4]:
f_visitors_df[f_visitors_df['国籍']=='米国']

,都道府県名,年,国籍,年間訪問者数,第1四半期,第2四半期,第3四半期,第4四半期
13,東京都,2012,米国,439900,100278,125351,97118,117153
28,東京都,2013,米国,510599,106241,147160,120904,136294
46,東京都,2014,米国,607617,128122,181661,140535,157299
66,東京都,2015,米国,700784,133039,207550,169506,190689
86,東京都,2016,米国,867692,158804,243956,210586,254346
106,東京都,2017,米国,1053497,209625,316580,246996,280296
126,東京都,2018,米国,1190658,242191,352172,275904,320391
146,東京都,2019,米国,1313048,262089,404908,315981,330070


例2. 国籍別訪問者数合計を計算する

In [11]:
f_visitors_byNation = f_visitors_df.groupby('国籍').sum(numeric_only=True)
f_visitors_byNation

,年,年間訪問者数,第1四半期,第2四半期,第3四半期,第4四半期
国籍,,,,,,
イタリア,10085,571217,93166,162505,182940,132606
インド,16124,589855,133038,179674,135122,142021
インドネシア,12099,1425495,305268,432704,251217,436306
オーストラリア,16124,2654234,710167,635899,543569,764599
カナダ,16124,1593411,350745,423221,373063,446382
シンガポール,16124,1722714,322323,463069,269432,667890
スペイン,10085,470073,69387,110276,177395,113015
タイ,16124,3504201,823686,1027845,510787,1141883
ドイツ,16124,1044196,226944,281021,254317,281914


例3. 年間訪問者数の大きい順にトップ10まで表示する

In [14]:
f_visitors_byNation=f_visitors_byNation.sort_values('年間訪問者数', ascending=False)
f_visitors_byNation.iloc[:10,:]

,年,年間訪問者数,第1四半期,第2四半期,第3四半期,第4四半期
国籍,,,,,,
中国,16124,24321128,5382810,5831901,7950827,5155590
台湾,16124,9721172,2265145,2666165,2592461,2197401
韓国,16124,8781276,2384443,2121762,2178078,2096993
米国,16124,6683795,1340389,1979338,1577530,1786538
香港,16124,4191802,959387,1079870,1078001,1074544
タイ,16124,3504201,823686,1027845,510787,1141883
オーストラリア,16124,2654234,710167,635899,543569,764599
英国,16124,1940831,434520,494494,474173,537644
シンガポール,16124,1722714,322323,463069,269432,667890


# 解説1. DataFrameはデータ分析のためのデータ形式
1. 行見出し、列見出しが付いた2次元配列　
1. 行、列を自由に追加・削除できる　
2. 特定の行、列を様々な条件で抽出できる
3. 表形式での計算が容易（行合計、列合計、小計など）


# DataFrameを作成する
1. 2次元配列を作る

In [4]:
import numpy as np

profiles =[['Lucy',160,52,'A','USA','20','student'],
 ['Bob',180,52,'O','England','30','dentist'],
 ['Shohei',193,80,'AB','Japan','30','MLB'],
 ['Mami',180,62,'B','Japan','28','wife']]
profiles

[['Lucy', 160, 52, 'A', 'USA', '20', 'student'],
 ['Bob', 180, 52, 'O', 'England', '30', 'dentist'],
 ['Shohei', 193, 80, 'AB', 'Japan', '30', 'MLB'],
 ['Mami', 180, 62, 'B', 'Japan', '28', 'wife']]

2. 見出しを付ける  
列見出し：データ項目が多くなると意味がわかるようにしたい  
行見出し：同名があるかもしれないので、idで識別したい

In [5]:
import pandas as pd
profile_df = pd.DataFrame(profiles, index=['U1','E1','J1','J2'],
                          columns = ['name','height','weight',
                                     'blood type','nation','age','occupation'])
profile_df

,name,height,weight,blood type,nation,age,occupation
U1,Lucy,160,52,A,USA,20,student
E1,Bob,180,52,O,England,30,dentist
J1,Shohei,193,80,AB,Japan,30,MLB
J2,Mami,180,62,B,Japan,28,wife



# 解説3. 行、列の追加

### 1) 列の追加
    構文：DataFrame名[追加する列名(column)]=[追加する列データ（データ数はDataFrameの行数と一致すること）]  
    例：列名 'male/female'  データ female, male, male, female を追加

In [6]:
profile_df['male/female']=['female', 'male', 'male', 'female']
profile_df

,name,height,weight,blood type,nation,age,occupation,male/female
U1,Lucy,160,52,A,USA,20,student,female
E1,Bob,180,52,O,England,30,dentist,male
J1,Shohei,193,80,AB,Japan,30,MLB,male
J2,Mami,180,62,B,Japan,28,wife,female


### 2) 列方向にDataFrameを連結する
    構文： 連結後DataFrame名 = pd.concat([DataFrame1, DataFrame2], axis = 1)   
    　　　 DataFrame1, DataFrame2の行数は一致していることが条件    
    例：2列のDataFrameをprofile_dfに連結する

In [7]:
profile2 = [['tennis','married'],
           ['game','single'],
           ['baseball','married'],
           ['baseball','married']]
profile_df2 = pd.DataFrame(profile2,index=['U1','E1','J1','J2'],columns=['hobby','married'])
profile_df2

,hobby,married
U1,tennis,married
E1,game,single
J1,baseball,married
J2,baseball,married


In [8]:
profile_df3 = pd.concat([profile_df,profile_df2],axis=1)
profile_df3

,name,height,weight,blood type,nation,age,occupation,male/female,hobby,married
U1,Lucy,160,52,A,USA,20,student,female,tennis,married
E1,Bob,180,52,O,England,30,dentist,male,game,single
J1,Shohei,193,80,AB,Japan,30,MLB,male,baseball,married
J2,Mami,180,62,B,Japan,28,wife,female,baseball,married


### 3). 行方向に連結する
    構文： 連結後DataFrame名 = pd.concat([DataFrame1, DataFrame2], axis = 0)  
          　DataFrame1, DataFrame2の列数は一致していることが条件
    例：2行のDataFrameをprofile_dfに連結する

In [9]:
profile4 = [['Frank',190,80,'O','Germany','15','student','male','ski','single'],
            ['Naomi',185,70,'A','Japan','30','tennis player','fimale','game','marreid']]
profile_df4 = pd.DataFrame(profile4, index=['FR','NA']
                           ,columns=['name','height','weight','blood type','nation','age',
                                     'occupation','male/female','hobby','married'])
profile_df4

,name,height,weight,blood type,nation,age,occupation,male/female,hobby,married
FR,Frank,190,80,O,Germany,15,student,male,ski,single
NA,Naomi,185,70,A,Japan,30,tennis player,fimale,game,marreid


In [16]:
profile_df5 = pd.concat([profile_df3,profile_df4],axis=0)
profile_df5

,name,height,weight,blood type,nation,age,occupation,male/female,hobby,married
U1,Lucy,160,52,A,USA,20,student,female,tennis,married
E1,Bob,180,52,O,England,30,dentist,male,game,single
J1,Shohei,193,80,AB,Japan,30,MLB,male,baseball,married
J2,Mami,180,62,B,Japan,28,wife,female,baseball,married
FR,Frank,190,80,O,Germany,15,student,male,ski,single
NA,Naomi,185,70,A,Japan,30,tennis player,fimale,game,marreid


# 解説4. スライシング
DataFrameから部分的な行・列範囲を取り出すことをスライシングと言う。

### 1) 行ラベル、列ラベルにもとづくスライシング
構文：DataFrame名[from行ラベルFrom : to行ラベル, from列ラベルFrom : to列ラベル]  
例1：上記のDataFrameから、行ラベル J1～FR かつ blood type～occupationの範囲をスライシングする



In [30]:
profile_df5.loc['J1':'FR',  'blood type':'occupation']

,blood type,nation,age,occupation
J1,AB,Japan,30,MLB
J2,B,Japan,28,wife
FR,O,Germany,15,student


例2：上記のDataFrameから、行ラベル J1～FR の全ての列をスライシングする

In [31]:
profile_df5.loc['J1':'FR', :]

,name,height,weight,blood type,nation,age,occupation,male/female,hobby,married
J1,Shohei,193,80,AB,Japan,30,MLB,male,baseball,married
J2,Mami,180,62,B,Japan,28,wife,female,baseball,married
FR,Frank,190,80,O,Germany,15,student,male,ski,single


例3：上記のDataFrameから、行ラベル J1～FR かつ列ラベル nation以降の全ての列をスライシングする

In [32]:
profile_df5.loc['J1':'FR','nation':]

,nation,age,occupation,male/female,hobby,married
J1,Japan,30,MLB,male,baseball,married
J2,Japan,28,wife,female,baseball,married
FR,Germany,15,student,male,ski,single


例4：上記のDataFrameから、行ラベル J1以降すべての行 かつ列ラベル name, nation, hobbyをスライシングする

In [33]:
profile_df5.loc['J1':,['name','nation','hobby']]

,name,nation,hobby
J1,Shohei,Japan,baseball
J2,Mami,Japan,baseball
FR,Frank,Germany,ski
NA,Naomi,Japan,game


### 2) 行インデックス、列インデックスにもとづくスライシング
構文：DataFrame名[from行インデックスFrom : to行インデックス, from列インデックスFrom : to列インデックス]  
例1：上記のDataFrameから、行ラベル J1～FR かつ blood type～occupationの範囲を行インデックス、列インデックスを使ってスライシングする

In [34]:
profile_df5.iloc[2:5,3:7]

,blood type,nation,age,occupation
J1,AB,Japan,30,MLB
J2,B,Japan,28,wife
FR,O,Germany,15,student


In [ ]:
例2：上記のDataFrameから、行ラベル J1～FR の全ての列を行インデックス、列インデックスを使ってスライシングする

In [35]:
profile_df5.iloc[2:5,:]

,name,height,weight,blood type,nation,age,occupation,male/female,hobby,married
J1,Shohei,193,80,AB,Japan,30,MLB,male,baseball,married
J2,Mami,180,62,B,Japan,28,wife,female,baseball,married
FR,Frank,190,80,O,Germany,15,student,male,ski,single


In [ ]:
例3: 上記のDataFrameから。3行目～4行目かつ1列目～4列目をスライシングする

In [36]:
profile_df5.iloc[2:4,0:4]

,name,height,weight,blood type
J1,Shohei,193,80,AB
J2,Mami,180,62,B


# 演習1 Dataframeの作成
以下のようなDataFrameを作成するコーディングを書け。なお、indexは名前の頭文字2桁とする。

In [18]:
import pandas as pd

scores = [['Alice',70,80,85],['Shohei',95,78,70],['Bob',65,85,97],['Charlie',90,95,100]]

df = pd.DataFrame(scores, index = ['AL','SH','BO','CH'],columns=['名前','国語','数学','英語'])
df



,名前,国語,数学,英語
AL,Alice,70,80,85
SH,Shohei,95,78,70
BO,Bob,65,85,97
CH,Charlie,90,95,100


# 演習2. 行、列の追加

1) 上記のDataFrameに以下の列を追加せよ。  
 列名：国籍、データ： ['アメリカ','イギリス','カナダ','イギリス']

In [19]:
df['国籍']=['アメリカ','イギリス','カナダ','イギリス']
df

,名前,国語,数学,英語,国籍
AL,Alice,70,80,85,アメリカ
SH,Shohei,95,78,70,イギリス
BO,Bob,65,85,97,カナダ
CH,Charlie,90,95,100,イギリス


2) 上記のDataFrameに以下の2列のDataFrameを連結せよ

In [23]:
profile=[['中1','10分'],['中2','30分'],['中1','15分'],['中3','5分']]
profile_df = pd.DataFrame(profile, index = ['AL','SH','BO','CH'],columns=['学年','通学時間'])
profile_df

,学年,通学時間
AL,中1,10分
SH,中2,30分
BO,中1,15分
CH,中3,5分


In [25]:
df2 = pd.concat([df,profile_df],axis = 1) 
df2

,名前,国語,数学,英語,国籍,学年,通学時間
AL,Alice,70,80,85,アメリカ,中1,10分
SH,Shohei,95,78,70,イギリス,中2,30分
BO,Bob,65,85,97,カナダ,中1,15分
CH,Charlie,90,95,100,イギリス,中3,5分


3) 上記のDataFrameに以下の2行のDataFrameを連結せよ。なお、2行のDataFrameのindex名は、名前の頭文字2文字とする。

In [27]:
students = [['Mamiko',80,75,90,'日本','中1','10分'],['Peter',65,85,90,'ドイツ','中2','15分']]

students_df = pd.DataFrame(students, index=['MA','PE'], columns=['名前','国語','数学','英語','国籍','学年','通学時間'])
students_df

,名前,国語,数学,英語,国籍,学年,通学時間
MA,Mamiko,80,75,90,日本,中1,10分
PE,Peter,65,85,90,ドイツ,中2,15分


In [29]:
df3 = pd.concat([df2,students_df],axis=0)
df3

,名前,国語,数学,英語,国籍,学年,通学時間
AL,Alice,70,80,85,アメリカ,中1,10分
SH,Shohei,95,78,70,イギリス,中2,30分
BO,Bob,65,85,97,カナダ,中1,15分
CH,Charlie,90,95,100,イギリス,中3,5分
MA,Mamiko,80,75,90,日本,中1,10分
PE,Peter,65,85,90,ドイツ,中2,15分
